In [22]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant

In [23]:
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import pandas as pd
from sklearn.model_selection import train_test_split

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding


from datasets import Dataset, DatasetDict

In [25]:
# !pip install datasets
# !pip install transformers

In [26]:
df = pd.read_csv("labeled_data.csv")
df.head()

,column_a,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [27]:
df.dropna(inplace = True)

In [28]:
df = df[['tweet','class']]

df.rename(columns ={'class':'label','tweet':'text'}, inplace = True)

In [29]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [30]:
df_train.shape

(19826, 2)

In [31]:
df_test.shape

(4957, 2)

In [32]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({'train': train_dataset, 'test': test_dataset})

In [33]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [35]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/19826 [00:00<?, ? examples/s]

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

In [36]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [37]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [38]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs = 5
)

Epoch 1/5
2479/2479 [==============================] - 451s 163ms/step - loss: 0.2879 - accuracy: 0.8999 - val_loss: 0.2421 - val_accuracy: 0.9106
Epoch 2/5
2479/2479 [==============================] - 383s 154ms/step - loss: 0.2039 - accuracy: 0.9264 - val_loss: 0.2478 - val_accuracy: 0.9155
Epoch 3/5
2479/2479 [==============================] - 372s 150ms/step - loss: 0.1538 - accuracy: 0.9438 - val_loss: 0.2824 - val_accuracy: 0.9052
Epoch 4/5
2479/2479 [==============================] - 373s 150ms/step - loss: 0.1039 - accuracy: 0.9617 - val_loss: 0.3357 - val_accuracy: 0.8959
Epoch 5/5
2479/2479 [==============================] - 384s 155ms/step - loss: 0.0596 - accuracy: 0.9797 - val_loss: 0.3994 - val_accuracy: 0.9030


In [39]:
from huggingface_hub import notebook_login

notebook_login()

In [40]:
model.push_to_hub("my-awesome-bert-model-hate-speech")
tokenizer.push_to_hub("my-awesome-bert-model-hate-speech")

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Guspfc/my-awesome-bert-model-hate-speech/commit/d8021ad8c2c6160f5784e1f7b7dd39b5912ec470', commit_message='Upload tokenizer', commit_description='', oid='d8021ad8c2c6160f5784e1f7b7dd39b5912ec470', pr_url=None, pr_revision=None, pr_num=None)